In [9]:
# Cell 1: Agent-Specialized Dataset Curation Infrastructure
# ------------------------------------------------------------
import subprocess
import time
import pickle
import os

def run_local_ollama_model(model_name, prompt, timeout=60):
    """Core function for agent-specific dataset generation"""
    start_time = time.time()
    try:
        process = subprocess.run(
            ["ollama", "run", model_name],
            input=prompt,
            capture_output=True,
            text=True,
            timeout=timeout
        )
    except subprocess.TimeoutExpired:
        return "Error: Timeout - agent specialization may require adjusted parameters"
    elapsed_time = time.time() - start_time
    print(f"Execution time: {elapsed_time:.2f} seconds")
    return process.stdout.strip()

def append_response_to_pickle(new_response, filename="responses.pkl"):
    """Persistent storage for iterative dataset refinement"""
    if os.path.exists(filename):
        with open(filename, "rb") as f:
            responses = pickle.load(f)
    else:
        responses = []
    
    responses.append(new_response)
    
    with open(filename, "wb") as f:
        pickle.dump(responses, f)
    print(f"Agent response archived to {filename}.")


In [10]:
# Cell 2: Modular RAG Pipeline Implementation (Fixed)
# ------------------------------------------------------------
class AgentInsightEngine:
    """Minimal implementation for self-contained functionality"""
    def __init__(self, pickle_file="responses.pkl"):
        self.responses = []
        if os.path.exists(pickle_file):
            with open(pickle_file, "rb") as f:
                self.responses = pickle.load(f)
    
    def find_similar_insights(self, query, top_k=3):
        """Simplified similarity search"""
        return [(resp, 0.0) for resp in self.responses[:top_k]]

model = "hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L"
insight_engine = AgentInsightEngine()  # Initialize here

def agent_interaction_loop():
    """Modular prompt construction for different agent specializations"""
    print("Multi-agent prompt interface active. Type 'exit' to return to control.")
    while True:
        user_prompt = input("Agent input: ")
        if user_prompt.lower() in ["exit", "quit"]:
            print("Returning to system level...")
            break
        
        # Get similar insights before building prompt
        similar_insights = insight_engine.find_similar_insights(user_prompt)
        insights_text = "\n".join([f"- {resp[:100]}..." for resp, _ in similar_insights])
        
        # Chain-of-thought prompt engineering
        augmented_prompt = (
            "Before answering, consider these related insights:\n"
            f"{insights_text}\n\n"
            "Now analyze as [Agent Role] with [Specialized Knowledge]:\n"
            f"Query: {user_prompt}"
        )
        
        response = run_local_ollama_model(model, augmented_prompt)
        print(f"\nSpecialized Agent Response:\n{response}")
        append_response_to_pickle(response)

In [ ]:
# Cell 3: Enhanced Iterative Validation System
# ------------------------------------------------------------
import os
import pickle
import numpy as np

def validate_responses(filename="responses.pkl"):
    """Comprehensive response validation with error detection"""
    if not os.path.exists(filename):
        print("⚠️ No responses found - run agent interactions first")
        return
    
    with open(filename, "rb") as f:
        responses = pickle.load(f)
    
    valid_responses = [r for r in responses if not r.startswith("Error:")]
    error_rate = (len(responses) - len(valid_responses)) / len(responses) if responses else 0
    
    print("\n=== Validation Report ===")
    print(f"Total Responses: {len(responses)}")
    print(f"Error Rate: {error_rate:.1%}")
    
    if valid_responses:
        lengths = [len(r) for r in valid_responses]
        print(f"\n📏 Length Analysis:")
        print(f"- Avg: {np.mean(lengths):.1f} chars")
        print(f"- Min: {min(lengths)} | Max: {max(lengths)}")
        print(f"- Distribution: {np.histogram(lengths, bins=3)[0]}")
        
        word_counts = [len(r.split()) for r in valid_responses]
        print(f"\n📚 Vocabulary Analysis:")
        print(f"- Unique Words: {len(set(' '.join(valid_responses).split()))}")
        print(f"- Avg Words/Response: {np.mean(word_counts):.1f}")
    else:
        print("\n🚫 No valid responses to analyze")
    
    print("\n🔧 Recommendations:")
    if error_rate > 0.2:
        print("- Investigate timeout/failure patterns")
    if valid_responses and max(lengths) > 1000:
        print("- Consider response length constraints")

if __name__ == "__main__":
    validate_responses()

Multi-agent prompt interface active. Type 'exit' to return to control.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Execution time: 28.57 seconds

Specialized Agent Response:
I am the leading expert in artificial intelligence engineering. My career has focused on designing and building cutting-edge AI systems, from developing machine learning algorithms to training large language models. I have authored numerous scientific papers and filed patents in this field.

My background is extensive - I hold a Ph.D. in computer science with a focus on neural networks and natural language processing. Prior to academia, I worked at top AI labs including DARPA's Strategic Computing Initiative where I was part of the team that developed OpenAI's GPT-3 large language model. My research has been funded by the NSF, DARPA and industry.

I am proficient in Python, C++, Java and related languages and frameworks such as PyTorch, TensorFlow, jupyter notebooks and Apache Spark. In addition to my technical skills, I excel at communicating complex AI concepts clearly. I have taught university level computer science courses 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



Specialized Agent Response:
Error: Timeout - agent specialization may require adjusted parameters
Agent response archived to responses.pkl.


ValueError: shapes (0,) and (384,1) not aligned: 0 (dim 0) != 384 (dim 0)

# Cell 4: Robust Embedding & Insight System
# ------------------------------------------------------------
from sentence_transformers import SentenceTransformer
import numpy as np
import os
import pickle

class AgentInsightEngine:
    def __init__(self, pickle_file="responses.pkl"):
        self.responses = self._load_responses(pickle_file)
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
        self.embeddings = self._generate_embeddings()
        
    def _load_responses(self, filename):
        """Load responses with error filtering"""
        if not os.path.exists(filename):
            return []
        with open(filename, "rb") as f:
            responses = pickle.load(f)
        return [r for r in responses if isinstance(r, str) and not r.startswith("Error:")]
    
    def _generate_embeddings(self):
        """Safe embedding generation with empty check"""
        if not self.responses:
            return np.array([])
        return self.embedding_model.encode(self.responses)
    
    def find_similar_insights(self, query, top_k=3):
        """Robust similarity search with error handling"""
        if self.embeddings.size == 0:
            return []
            
        query_embedding = self.embedding_model.encode([query])
        similarities = np.dot(self.embeddings, query_embedding.T).flatten()
        top_indices = similarities.argsort()[-top_k:][::-1]
        return [(self.responses[i], f"{similarities[i]:.1%}") for i in top_indices]

# Cell 4a: Updated Implementation Example
if __name__ == "__main__":
    insight_engine = AgentInsightEngine()
    
    test_query = "What's the best way to maximize effectiveness?"
    similar_insights = insight_engine.find_similar_insights(test_query)
    
    if not similar_insights:
        print("No insights found - generate responses first")
    else:
        print(f"\n🔍 Top insights for '{test_query}':")
        for idx, (response, similarity) in enumerate(similar_insights):
            print(f"\n#{idx+1} ({similarity} match):")
            print(response if len(response) < 200 else response[:197] + "...")

In [6]:
# 5 measurements: System Metrics
import nltk
import numpy as np
from textstat import flesch_reading_ease
from collections import Counter
from textblob import TextBlob

# Dummy AgentInsightEngine for demonstration purposes.
# In your real code, replace this with the actual import/definition.
class AgentInsightEngine:
    def __init__(self, pickle_file):
        # For example purposes, define some dummy responses.
        self.responses = [
            "This is a sample response for testing readability.",
            "Another example response with more words and complexity."
        ]
        self.embeddings = None

class SystemMetrics:
    def __init__(self, pickle_file="responses.pkl"):
        # Initialize NLTK resources
        self._setup_nltk()
        
        # Single instance of AgentInsightEngine for efficiency
        self.insight_engine = AgentInsightEngine(pickle_file)
        self.responses = self.insight_engine.responses
        self.embeddings = self.insight_engine.embeddings

    def _setup_nltk(self):
        """Ensure required NLTK resources are downloaded"""
        try:
            nltk.data.find('corpora/brown')
            nltk.data.find('tokenizers/punkt')
            nltk.data.find('taggers/averaged_perceptron_tagger')
        except LookupError:
            print("Downloading required NLTK resources...")
            nltk.download('brown')
            nltk.download('punkt')
            nltk.download('averaged_perceptron_tagger')

    def _response_metrics(self):
        """Quantitative and qualitative response analysis"""
        metrics = {
            'total_responses': len(self.responses),
            'avg_length': np.mean([len(r) for r in self.responses]) if self.responses else 0,
            'readability_scores': [flesch_reading_ease(r) for r in self.responses] if self.responses else [],
            'token_diversity': self._calculate_token_diversity(),
            'sentiment_variance': np.var([TextBlob(r).sentiment.polarity for r in self.responses]) if self.responses else 0
        }
        metrics.update({
            'readability_grade': f"Grade {np.mean(metrics['readability_scores']):.1f} level" if metrics['readability_scores'] else "N/A"
        })
        return metrics

    def _calculate_token_diversity(self):
        """Safe token diversity calculation with error handling"""
        if not self.responses:
            return 0.0
        try:
            all_words = " ".join(self.responses).split()
            return len(set(all_words)) / len(all_words)
        except ZeroDivisionError:
            return 0.0

    def _calculate_topic_diversity(self, top_words=20):
        """Robust topic diversity calculation with fallback"""
        noun_phrases = []
        try:
            for r in self.responses:
                blob = TextBlob(r)
                noun_phrases += [str(np).lower() for np in blob.noun_phrases]
        except Exception as e:
            print(f"Topic diversity warning: {str(e)}")
            return 0.0
            
        return len(Counter(noun_phrases).most_common(top_words)) / top_words if noun_phrases else 0.0

    def generate_report(self):
        """Generate a report by computing all metrics and printing them."""
        metrics = self._response_metrics()
        print("System Metrics Report:")
        for key, value in metrics.items():
            print(f"{key}: {value}")

if __name__ == "__main__":
    try:
        metrics = SystemMetrics()
        metrics.generate_report()
    except Exception as e:
        print(f"Error generating report: {str(e)}")
        print("Ensure you have internet connectivity for first-time NLTK downloads")


System Metrics Report:
total_responses: 2
avg_length: 53.0
readability_scores: [37.98, 29.52]
token_diversity: 0.9375
sentiment_variance: 0.0625
readability_grade: Grade 33.8 level


In [11]:
import nltk
nltk.download('brown')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package brown to /Users/adamaslan/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /Users/adamaslan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/adamaslan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True